In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset


In [2]:
# Load Dataset
data = pd.read_csv('winequality-white.csv', delimiter=';')


In [3]:
# Preprocess Data
X = data.drop('quality', axis=1).values
y = data['quality'].values


In [4]:
# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [5]:
# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [6]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Convert to PyTorch Tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [8]:
# Define RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, pooling_type):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.pooling_type = pooling_type
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        if self.pooling_type == 'max':
            pooled, _ = torch.max(rnn_out, dim=1)
        elif self.pooling_type == 'avg':
            pooled = torch.mean(rnn_out, dim=1)
        else:
            raise ValueError("Pooling type must be 'max' or 'avg'")
        out = self.fc(pooled)
        return out


In [9]:
# Define Deep RNN Model
class DeepRNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, pooling_type, num_layers):
        super(DeepRNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.pooling_type = pooling_type
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        if self.pooling_type == 'max':
            pooled, _ = torch.max(rnn_out, dim=1)
        elif self.pooling_type == 'avg':
            pooled = torch.mean(rnn_out, dim=1)
        else:
            raise ValueError("Pooling type must be 'max' or 'avg'")
        out = self.fc(pooled)
        return out

In [25]:
# Training and Evaluation Function with Optimized Early Stopping and Mid-Epoch Break
def train_and_evaluate(
    model,
    optimizer,
    criterion,
    train_loader,
    test_loader,
    num_epochs,
    scheduler=None,
    early_stopping_patience=5,  # Reduce patience
    tolerance=1e-4,
    min_improvement=1e-4,
    max_epoch_limit=12,  #limit to stop before halfway if specified
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available
):
    model.to(device)
    best_acc = 0
    best_loss = float('inf')
    patience_counter = 0
    recent_accuracies = []
    recent_losses = []

    # Calculate mid-epoch if no limit is specified
    if max_epoch_limit is None:
        max_epoch_limit = num_epochs // 2

    for epoch in range(num_epochs):
        # Stop if reaching the epoch limit
        if epoch > max_epoch_limit:
            print(f"Stopped early at epoch {epoch + 1} due to reaching max_epoch_limit .")
            break

        model.train()
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch.unsqueeze(1))
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        if scheduler:
            scheduler.step()

        # Evaluation
        model.eval()
        y_pred = []
        y_true = []
        test_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch.unsqueeze(1))
                loss = criterion(outputs, y_batch)
                test_loss += loss.item()
                _, preds = torch.max(outputs, dim=1)
                y_pred.extend(preds.cpu().numpy())  # Move to CPU for metrics
                y_true.extend(y_batch.cpu().numpy())  # Move to CPU for metrics
        acc = accuracy_score(y_true, y_pred)
        avg_test_loss = test_loss / len(test_loader)

        # Update recent metrics
        recent_accuracies.append(acc)
        recent_losses.append(avg_test_loss)
        if len(recent_accuracies) > early_stopping_patience:
            recent_accuracies.pop(0)
            recent_losses.pop(0)

        # Check for improvements in both accuracy and loss
        if acc > best_acc + tolerance or avg_test_loss < best_loss - tolerance:
            best_acc = max(best_acc, acc)
            best_loss = min(best_loss, avg_test_loss)
            patience_counter = 0
        elif len(recent_accuracies) >= early_stopping_patience:
            # Check if both accuracy and loss have plateaued
            if (
                all(abs(a - best_acc) < tolerance for a in recent_accuracies)
                and all(abs(l - best_loss) < tolerance for l in recent_losses)
            ):
                print(f"Early stopping at epoch {epoch + 1} due to stable performance (accuracy and loss).")
                break

            # Check if minimum improvement threshold is not met
            if (
                max(recent_accuracies) - min(recent_accuracies) < min_improvement
                and max(recent_losses) - min(recent_losses) < min_improvement
            ):
                print(f"Early stopping at epoch {epoch + 1} due to lack of significant improvement.")
                break
        else:
            patience_counter += 1

        if patience_counter >= early_stopping_patience:
            print(f"Early stopping at epoch {epoch + 1} due to patience limit.")
            break

        print(f"Epoch {epoch + 1}/{num_epochs}, Accuracy: {acc:.4f}, Loss: {avg_test_loss:.4f}")

    return best_acc


In [26]:
# Experiment Parameters
hidden_sizes = [16, 32, 64]
pooling_types = ['max', 'avg']
num_epochs_list = [5, 50, 100, 250, 350]
optimizers = [optim.SGD, optim.RMSprop, optim.Adam]

input_size = X_train.shape[1]
output_size = len(np.unique(y_train))

results = []


In [27]:
# Run Experiments for RNN and Deep RNN
for model_type in ['RNN', 'DeepRNN']:
    for hidden_size in hidden_sizes:
        for pooling_type in pooling_types:
            for num_epochs in num_epochs_list:
                for optimizer_fn in optimizers:
                    print(f"Running: model={model_type}, hidden_size={hidden_size}, pooling={pooling_type}, epochs={num_epochs}, optimizer={optimizer_fn.__name__}")

                    if model_type == 'RNN':
                        model = RNNModel(input_size, hidden_size, output_size, pooling_type)
                    elif model_type == 'DeepRNN':
                        model = DeepRNNModel(input_size, hidden_size, output_size, pooling_type, num_layers=3)

                    optimizer = optimizer_fn(model.parameters(), lr=0.01)
                    criterion = nn.CrossEntropyLoss()

                    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

                    best_acc = train_and_evaluate(
                        model, optimizer, criterion, train_loader, test_loader, num_epochs, scheduler
                    )

                    results.append({
                        'model': model_type,
                        'hidden_size': hidden_size,
                        'pooling': pooling_type,
                        'epochs': num_epochs,
                        'optimizer': optimizer_fn.__name__,
                        'accuracy': best_acc
                    })

# Convert Results to DataFrame
results_df = pd.DataFrame(results)
print(results_df)

# Save Results to CSV
results_df.to_csv('experiment_results_1.csv', index=False)

Running: model=RNN, hidden_size=16, pooling=max, epochs=5, optimizer=SGD
Epoch 1/5, Accuracy: 0.4857, Loss: 1.5577
Epoch 2/5, Accuracy: 0.4939, Loss: 1.3737
Epoch 3/5, Accuracy: 0.5000, Loss: 1.2825
Epoch 4/5, Accuracy: 0.5010, Loss: 1.2332
Epoch 5/5, Accuracy: 0.5000, Loss: 1.2025
Running: model=RNN, hidden_size=16, pooling=max, epochs=5, optimizer=RMSprop
Epoch 1/5, Accuracy: 0.5296, Loss: 1.0976
Epoch 2/5, Accuracy: 0.5439, Loss: 1.0707
Epoch 3/5, Accuracy: 0.5327, Loss: 1.0713
Epoch 4/5, Accuracy: 0.5367, Loss: 1.0771
Epoch 5/5, Accuracy: 0.5194, Loss: 1.0824
Running: model=RNN, hidden_size=16, pooling=max, epochs=5, optimizer=Adam
Epoch 1/5, Accuracy: 0.5306, Loss: 1.0993
Epoch 2/5, Accuracy: 0.5184, Loss: 1.0931
Epoch 3/5, Accuracy: 0.5245, Loss: 1.0902
Epoch 4/5, Accuracy: 0.5480, Loss: 1.0718
Epoch 5/5, Accuracy: 0.5418, Loss: 1.0580
Running: model=RNN, hidden_size=16, pooling=max, epochs=50, optimizer=SGD
Epoch 1/50, Accuracy: 0.4490, Loss: 1.5728
Epoch 2/50, Accuracy: 0.4714,